In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Online algorithm for the mean

Univariate

In [ ]:
def update_mean(prev_mean,new_data,n):
    new_mean = prev_mean + 1/float(n)*(new_data-prev_mean)
    return new_mean

In [ ]:
print update_mean(0,1,1)
print update_mean(1,0,2)


In [ ]:
values = np.random.randint(0,9,size=(10000))

In [ ]:
values


In [ ]:
values.mean()

In [ ]:
prev_mean = values[0]
n = 1
for new_data in values[1:]:
    n+=1
    new_mean = update_mean(prev_mean,new_data,n)
    prev_mean = new_mean
    print new_mean 
    

In [ ]:
def rolling_expand(arr):
    assert len(arr.shape) == 1, "Array must be 1-dimensional"
    padded_array = np.concatenate([np.nan*np.empty(shape=arr.shape[0]-1),arr],axis=0)
    return np.lib.stride_tricks.as_strided(padded_array,shape=(arr.shape[0],arr.shape[0]),strides=(8,8),writeable=False)
    

In [ ]:
rolling_means = np.nanmean(rolling_expand(values),axis=0)

## Online covariance

In [ ]:
def update_cov_matrix(cov,xn,xn_mean,xn1_mean):
    return cov + np.matmul((xn-xn_mean)[None,:].transpose(),(xn-xn1_mean)[None,:])

In [ ]:
real_cov = np.array([[5,0.5],[0.5,1]])
real_mean = np.array([0,10])

In [ ]:
real_cov, real_mean

In [ ]:
N = 1000

In [ ]:
x = np.random.multivariate_normal(real_mean,real_cov,size=N)
estimated_s = 0
prev_mean = x[0]
for i, xi in enumerate(x[1:]):
    new_mean = update_mean(prev_mean,xi,i+1)
    estimated_s = update_cov_matrix(estimated_s,xi,new_mean,prev_mean)
    prev_mean = new_mean
    
estimated_s/float(N)

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(x[:,0],x[:,1])

## Draft for outlier detector algo

**Formula for online mean update:**

$ \bar{X}_n = \bar{X}_{n-1} + (X_n - \bar{X}_{n-1}) $

**Formula for online Covariance Matrix Estimator update:**

$ S_n = S_{n-1} + (X_n - \bar{X}_n)(X_n - \bar{X}_{n-1})^t $

$ C_n = \frac{1}{n-1}S_n $

In [ ]:
class MockSelf(object):
    pass
self = MockSelf()

In [ ]:
# Ground truth

real_cov = np.array([[5,2],[2,1]])
real_mean = np.array([0,10])

In [ ]:
p = 10
tmp_mat = np.random.normal(0,2,size=(p,p))
real_cov = np.matmul(tmp_mat.transpose(),tmp_mat)
real_mean = np.random.normal(0,5,size=p)

In [ ]:
# state
self.mean = None
self.n = 0
self.C = 0

In [ ]:
# inputs
features = np.random.multivariate_normal(real_mean,real_cov,size=1)
feature_names = []

# score function

self.n += 1
feat0 = features[0]

if self.mean is None:
    self.mean = feat0
    assert False, 0
else:
    new_mean = self.mean + 1./float(self.n)*(feat0 - self.mean)
    self.C = self.C + np.matmul((feat0 - self.mean)[None,:].transpose(),(feat0 - new_mean)[None,:])
    self.mean = new_mean
    
mean = self.mean
cov = self.C/float(self.n-1)

cov1 = np.linalg.inv(cov)

# Mahalanobis distance = (X-u)t * C-1 * (X-u)
d2 = np.matmul(feat0 - mean, np.matmul(feat0 - mean, cov1.transpose()).transpose())

print features, d2

## Handling Batches

** Formula for online batch mean update (batch of size $b$): **

$ \bar{X}_{n+b} = \bar{X}_n + \frac{b}{n+b}(\bar{X}_{n,n+b} - \bar{X}_n) $

where $\bar{X}_{n,n+b}$ is the mean of the batch.

**Theorem:**

if $A$ and $A+B$ are invertible and $rank(B) = 1$ then

$ (A + B)^{-1} = A^{-1} - \frac{1}{1+trace(BA^{-1})}A^{-1}BA^{-1} $

In our case is B of rank 1 ? Yes:

$ B = (X_n - \bar{X}_n)*(X_n - \bar{X}_{n-1}) $

and 

$ \bar{X}_n = \bar{X}_{n-1} * \frac{n-1}{n} + X_n * \frac{1}{n} $

$ X_n - \bar{X}_n = X_n * ( 1 - \frac{1}{n} ) - \bar{X}_{n-1} * \frac{n-1}{n} $

$ X_n = \frac{n-1}{n}(X_n - \bar{X}_{n-1}) $

So B is produced from a single vector, its rank can only be one

In [ ]:
self.n = 0 
self.mean = 0
self.C = 0

In [ ]:
# inputs
features = np.random.multivariate_normal(real_mean,real_cov,size=1000)
feature_names = []

nb = features.shape[0]

roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))

coefs = (np.arange(nb)+1.)/(np.arange(nb)+self.n+1.)

new_means = self.mean + coefs.reshape((nb,1))*(roll_partial_means-self.mean)

# lets compute rolling (Xn-mXn)t*(Xn-mXn-1)

new_means_offset = np.empty_like(new_means)
new_means_offset[0] = self.mean
new_means_offset[1:] = new_means[:-1]

B = np.matmul((features - new_means)[:,:,None],(features - new_means_offset)[:,None,:])

all_C = self.C + B.cumsum(axis=0)
all_C_inv = np.zeros_like(B)

all_C = np.roll(all_C,1,axis=0)
all_C[0] = self.C

c_inv = None
EPSILON = 1e-8

for i, b in enumerate(B):
    if c_inv is None:
        if abs(np.linalg.det(all_C[i])) > EPSILON:
            c_inv = np.linalg.inv(all_C[i])
        else:
            continue
    else:
        c_inv = all_C_inv[i-1]
    BC1 = np.matmul(b,c_inv)
    all_C_inv[i] = c_inv - 1./(1.+np.trace(BC1))*np.matmul(c_inv,BC1)

all_C_inv *= np.arange(nb).reshape((nb,1,1)) + self.n

self.C += B.sum(axis=0)

self.mean = new_means[-1]

self.n += nb

feat_diff = features-new_means_offset
outlier_scores = np.matmul(feat_diff[:,None,:],np.matmul(all_C_inv,feat_diff[:,:,None])).reshape(nb)

In [ ]:
plt.plot(outlier_scores)

In [ ]:
import OutlierZscore
reload(OutlierZscore)
from OutlierZscore import OutlierZscore

In [ ]:
detector = OutlierZscore()

In [ ]:
detector.score(features,[])

## PCA

In [ ]:
cov = self.C/(self.n-1.)

In [ ]:
cov

In [ ]:
plt.scatter(features[:,0],features[:,1])

In [ ]:
np.linalg.eigh?

In [ ]:
from scipy.linalg import eigh

In [ ]:
nd = cov.shape[0]

In [ ]:
n_vals = 1

In [ ]:
eigvals, eigvects = eigh(cov,eigvals=(nd-n_vals,nd-1))

In [ ]:
eigvals

In [ ]:
eigvects

In [ ]:
P = np.matmul(eigvects,eigvects.transpose())

In [ ]:
P

In [ ]:
normed_features = np.matmul(features,P.transpose())

In [ ]:
plt.scatter(normed_features[:,0],normed_features[:,1])

In [ ]:
# inputs
features = np.random.multivariate_normal(real_mean,real_cov,size=1000)
feature_names = []


In [ ]:
nb = features.shape[0]

roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))

coefs = (np.arange(nb)+1.)/(np.arange(nb)+self.n+1.)

new_means = self.mean + coefs.reshape((nb,1))*(roll_partial_means-self.mean)

# lets compute rolling (Xn-mXn)t*(Xn-mXn-1)

new_means_offset = np.empty_like(new_means)
new_means_offset[0] = self.mean
new_means_offset[1:] = new_means[:-1]

B = np.matmul((features - new_means)[:,:,None],(features - new_means_offset)[:,None,:])

In [ ]:
cov_complete_batch = (self.C + B.sum(axis=0))/(self.n+nb-1.)

In [ ]:
p = features.shape[1]

In [ ]:
n_components = 1

In [ ]:
cov_complete_batch

In [ ]:
eigvals, eigvects = eigh(cov_complete_batch,eigvals=(p-n_components,p-1))

proj_matrix = np.matmul(eigvects,eigvects.transpose())

proj_features = np.matmul(features,proj_matrix.transpose())

In [ ]:
plt.scatter((features - new_means[-1])[:,0],(features - new_means[-1])[:,1])
plt.scatter((proj_features-proj_features.mean(axis=0))[:,0],(proj_features-proj_features.mean(axis=0))[:,1])

In [ ]:
plt.scatter(proj_features[:,0],np.zeros(nb))

In [ ]:
proj_features.mean(axis=0)

In [ ]:
features = proj_features[:,:n_components]

In [ ]:
roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))

coefs = (np.arange(nb)+1.)/(np.arange(nb)+self.n+1.)

new_means = self.mean + coefs.reshape((nb,1))*(roll_partial_means-self.mean)

new_means_offset = np.empty_like(new_means)
new_means_offset[0] = self.mean
new_means_offset[1:] = new_means[:-1]

B = np.matmul((features - new_means)[:,:,None],(features - new_means_offset)[:,None,:])

all_C = self.C + B.cumsum(axis=0)
all_C_inv = np.zeros_like(B)

all_C = np.roll(all_C,1,axis=0)
all_C[0] = self.C

c_inv = None
EPSILON = 1e-8

for i, b in enumerate(B):
    if c_inv is None:
        if abs(np.linalg.det(all_C[i])) > EPSILON:
            c_inv = np.linalg.inv(all_C[i])
        else:
            continue
    else:
        c_inv = all_C_inv[i-1]
    BC1 = np.matmul(b,c_inv)
    all_C_inv[i] = c_inv - 1./(1.+np.trace(BC1))*np.matmul(c_inv,BC1)

all_C_inv *= np.arange(nb).reshape((nb,1,1)) + self.n

self.C += B.sum(axis=0)

self.mean = new_means[-1]

self.n += nb

feat_diff = features-new_means_offset
outlier_scores = np.matmul(feat_diff[:,None,:],np.matmul(all_C_inv,feat_diff[:,:,None])).reshape(nb)

In [ ]:
plt.plot(outlier_scores)

### Improved algo for online covariance

In [ ]:
real_cov = np.array([[5,2.],[2.,1]])
real_mean = np.array([0,10])

N = 1000

p = 2

X = np.random.multivariate_normal(real_mean,real_cov,N)

In [ ]:
np.cov(*X.transpose())

In [ ]:
X_ = np.cumsum(X,axis=0)/(np.arange(N).reshape((N,1))+1.)

In [ ]:
diffs = (X[1:]-X_[1:]).reshape((N-1,X.shape[1],1))
diffs_offset =  (X[1:]-np.roll(X_,1)[1:]).reshape((N-1,1,X.shape[1]))

In [ ]:
B = np.matmul(diffs,diffs_offset)

In [ ]:
B

In [ ]:
B = np.matmul((X-X_).reshape((N,X.shape[1],1)),(X-X_).reshape((N,1,X.shape[1])))

In [ ]:
B.shape

In [ ]:
B.sum(axis=0)/(N-1)

In [ ]:
np.roll(X_,1)

In [ ]:
X_

In [ ]:
X

In [ ]:
X_

In [ ]:
cX = (X-X_).reshape((N,1,X.shape[1]))

In [ ]:
B = np.matmul(cX.transpose((0,2,1)),cX)

In [ ]:
S = B.cumsum(axis=0)

In [ ]:
S = np.empty(shape=(N,p,p))

In [ ]:
for i in range(N):
    cX = (X[:i+1]-X_[i]).reshape(((i+1,1,X.shape[1])))
    S[i] = np.matmul(cX.transpose((0,2,1)),cX).sum(axis=0)

In [ ]:
S[-1]

In [ ]:
np.cov?

In [ ]:
S[-1]

In [ ]:
S[-2] + np.matmul((X[-1]-X_[-1])[:,None],(X[-1]-X_[-2])[None,:])

In [ ]:
S[-1] - S[-2]

In [ ]:
np.matmul((X[-1]-X_[-1])[:,None],(X[-1]-X_[-2])[None,:])

In [ ]:
np.cov(*X.transpose())*(len(X)-1) - np.cov(*X[:-1].transpose())*(len(X[:-1])-1)

In [ ]:
np.cov(*X[:-1].transpose())*(len(X[:-1])-1)

In [ ]:
S[-2]

In [ ]:
np.cov(X.transpose())*(len(X)-1)

In [ ]:
np.cov(*X.transpose())*(len(X)-1)

In [ ]:
C = S/np.arange(N).reshape((N,1,1))

In [ ]:
C[-1]

In [ ]:
S[-1]/(N-1)

In [ ]:
1./float(N-1)*(C[-2]-float(N-1)/float(N)*np.matmul((X[-1]-X_[-2])[:,None],(X[-1]-X_[-2])[None,:]))

In [ ]:
C[-2]-C[-1]

In [ ]:
(N-1)/float(N)*np.matmul((X[-1]-X_[-2])[:,None],(X[-1]-X_[-2])[None,:])

In [ ]:
C[-2]

In [ ]:
C[0]

In [ ]:
C[1]

Setting up the algorithm in an efficient way

In [ ]:
self.C

In [ ]:
features


In [ ]:
1./float(N-1)*(C[-2]-float(N-2)/float(N-1)*np.matmul((X[-1]-X_[-2])[:,None],(X[-1]-X_[-2])[None,:]))

In [ ]:
features = X[:100]

In [ ]:
self.mean = 0
self.C = 0

In [ ]:
for i in len(features):
    self.C = self

In [ ]:
self.mean = features[0]

In [ ]:
i = 2
n = i+1
x = features[i]

In [ ]:
self.C = self.C - 1./(n-1.)*(self.C - (n-1.)/n*np.matmul((x-self.mean)[:,None],(x-self.mean)[None,:]))

In [ ]:
self.mean = self.mean + 1./float(n)*(x - self.mean)

In [ ]:
self.C

In [ ]:
np.cov(*features[:i-1].transpose())

In [ ]:
features[:i]

In [ ]:
np.cov(features[:i+1].transpose())

In [ ]:
self.mean

In [ ]:
self.mean = np.mean(features[:i+1],axis=0)

In [ ]:
self.C = np.cov(features[:i+1].transpose())
self.mean = np.mean(features[:i+1],axis=0)

In [ ]:
self.C

In [ ]:
self.mean

In [ ]:
self.C

In [ ]:
np.cov(features[:i+1].transpose())

In [ ]:
i = 2
n = i+1
x = features[i]

self.C = np.cov(features[:i+1].transpose())
self.mean = np.mean(features[:i+1],axis=0)

i = 3
n = i+1
x = features[i]

new_C = self.C - 1./(n-1.)*(self.C - (n-1.)/n*np.matmul((x-self.mean)[:,None],(x-self.mean)[None,:]))

new_C_real = np.cov(features[:i+1].transpose())

diff_real = new_C_real - self.C
diff_observed = - 1./(n-1.)*(self.C - (n-1.)/n*np.matmul((x-self.mean)[:,None],(x-self.mean)[None,:]))

print "real"
print new_C_real
print "observed"
print new_C

In [ ]:
real_cov = np.array([[5,2.],[2.,1]])
real_mean = np.array([0,10])

N = 100

p = 2

features = np.random.multivariate_normal(real_mean,real_cov,N)

In [ ]:
n_components = 3

self.mean = 0
self.complete_mean = 0
self.C = 0
self.complete_C = 0
self.n = 0
self.n_components = n_components

In [ ]:
nb = features.shape[0] # batch size
p = features.shape[1] # number of features
n_components = min(self.n_components,p)

# PCA
roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))
coefs = (np.arange(nb)+1.)/(np.arange(nb)+self.n+1.)
new_means = self.complete_mean + coefs.reshape((nb,1))*(roll_partial_means-self.complete_mean)
new_means_offset = np.empty_like(new_means)
new_means_offset[0] = self.complete_mean
new_means_offset[1:] = new_means[:-1]

B = np.matmul((features - new_means_offset)[:,:,None],(features - new_means_offset)[:,None,:])
cov_complete_batch = (self.complete_C + B.sum(axis=0))/(self.n+nb-1.)

Let's test the following formula:

$ C_n = \frac{n-1-k}{n-1}C_{n-k} + \frac{1}{n-1}\sum^k_{i=1}\frac{n-i}{n-i+1}(X_{n-i+1}-\bar{X}_{n-i})(X_{n-i+1}-\bar{X}_{n-i})^t$  
$\forall k \in [1,n-1]$

or

$ C_{n+k} = \frac{n-1}{n+k-1}C_{n} + \frac{1}{n+k-1}\sum^{k-1}_{i=0}\frac{n+i}{n+i+1}(X_{n+i+1}-\bar{X}_{n+i})^2 $

or

$ C_{N} = \frac{n-1}{N-1}C_{n} + \frac{1}{N-1}\sum^{N-1}_{i=n}\frac{i}{i+1}(X_{i+1}-\bar{X}_{i})^2 $


In [ ]:
B.shape

In [ ]:
coefs = ((self.n+np.arange(nb))/(self.n+np.arange(nb)+1.)).reshape((nb,1,1))

In [ ]:
B = coefs*np.matmul((features - new_means_offset)[:,:,None],(features - new_means_offset)[:,None,:])

In [ ]:
cov_complete_batch = (self.n-1.)/(self.n+nb-1.)*self.C + 1./(self.n+nb-1.)*B.sum(axis=0)

In [ ]:
cov_complete_batch

In [ ]:
np.cov(features.transpose())

In [ ]:
X = np.random.multivariate_normal(real_mean,real_cov,N*2)


In [ ]:
i = 1
features = X[i*N:(i+1)*N]

In [ ]:
nb = features.shape[0] # batch size
p = features.shape[1] # number of features
n_components = min(self.n_components,p)

# PCA
roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))
coefs = (np.arange(nb)+1.)/(np.arange(nb)+self.n+1.)
new_means = self.complete_mean + coefs.reshape((nb,1))*(roll_partial_means-self.complete_mean)
new_means_offset = np.empty_like(new_means)
new_means_offset[0] = self.complete_mean
new_means_offset[1:] = new_means[:-1]

coefs = ((self.n+np.arange(nb))/(self.n+np.arange(nb)+1.)).reshape((nb,1,1))
B = coefs*np.matmul((features - new_means_offset)[:,:,None],(features - new_means_offset)[:,None,:])
cov_complete_batch = (self.n-1.)/(self.n+nb-1.)*self.complete_C + 1./(self.n+nb-1.)*B.sum(axis=0)

In [ ]:
cov_complete_batch

In [ ]:
np.cov(X.transpose())

In [ ]:
self.n += nb
self.complete_mean = new_means[-1]
self.complete_C = cov_complete_batch